In [1]:
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords 
import re
from unicodedata import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
teste = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Cópia de Data_Test.csv')
treino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Cópia de Data_Train.csv')

In [5]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773 entries, 0 to 1772
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       1773 non-null   int64 
 1   Message  1773 non-null   object
 2   SPAM     1773 non-null   bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 29.6+ KB


In [6]:
treino.groupby(['SPAM']).count()

,ID,Message
SPAM,,
False,841,841
True,932,932


In [11]:
def pre_processamento(df, coluna):
  '''
  Retira stop words, normaliza retirando acentos e emojis, retira caracteres 
  especiais e pontuação, converte para minúsculo e transforma texto corrido em 
  listas de palavras.
  '''
  #Definindo variáveis e objetos
  message_list = []
  stop_words = [normalize('NFKD', word.encode('latin-1', 'ignore').decode('latin-1')) for word in set(stopwords.words('portuguese'))]
  regex = re.compile('[0-9]|\,|\.|\\n|\$|\(|\)|-|\+|\:|\;|\•|\?|\!|\&quot|\&|\#|\_')
  lemmatizer = WordNetLemmatizer()
  stemmer = PorterStemmer()
  #Retirando pontuação, emojis e caracteres especiais
  df[coluna] = df[coluna].apply(lambda x: re.sub(regex, ' ', normalize('NFKD', x.lower().encode('latin-1', 'ignore').decode('latin-1'))).split(' '))
  for lista in df[coluna]:
    #Retirando stop_words, caracteres vazios, letras únicas que não formam palavras e lematizando
    temp = [stemmer.stem(lemmatizer.lemmatize(palavra)) for palavra in lista if palavra != '' and len(palavra) > 1 and palavra not in stop_words]
    temp = ' '.join(temp)
    message_list.append(temp)
  #Preenchendo o dataframe de retorno com o resultado
  df[coluna] = message_list
  return df[coluna]

In [13]:
treino['Message'] = pre_processamento(treino, 'Message')

In [14]:
treino.head()

,ID,Message,SPAM
0,615317,linda msg,True
1,855344,to fazendo batida kwui coco,True
2,1007222,dinheiro vai sair nome lista dinheiro vai sair...,True
3,811063,msm irmã nao lembrava nome kkk,True
4,913944,ai xara sujiro,True


In [20]:
tf_idf_objeto = TfidfVectorizer(ngram_range = (1,2))
matrix_esparca = tf_idf_objeto.fit_transform(treino['Message'])

In [26]:
be

<1773x34357 sparse matrix of type '<class 'numpy.float64'>'
	with 57047 stored elements in Compressed Sparse Row format>

In [25]:
for numero in be[0].toarray()[0]:
  if numero != 0.0:
    print(numero)

0.49970828405337586
0.6741191368901676
0.5439255648771125
